In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('NER_Dataset.csv',error_bad_lines=False,engine='python')

<ipython-input-2-b3b3649ef103>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('NER_Dataset.csv',error_bad_lines=False,engine='python')


In [ ]:
df.shape

(47959, 4)

In [ ]:
df.head()

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."


In [ ]:
df.isnull().sum()

Sentence_ID    0
Word           0
POS            0
Tag            0
dtype: int64

In [ ]:
x = list(df['Word'])
y = list(df['Tag'])

In [ ]:
df['splited_word'] = df['Word']

In [ ]:
print(type(y))

<class 'list'>


### Pre processing Tags

In [ ]:
def preprocess_data(df):
    for i in range(len(df)):
        pos = ast.literal_eval(df['POS'][i])
        tags = ast.literal_eval(df['Tag'][i])
        word = ast.literal_eval(df['Word'][i])
        df['POS'][i] = [str(word) for word in pos]
        df['Tag'][i] = [str(word.upper()) for word in tags]
        df['Word'][i] = [str(word) for word in word]
    return df

In [ ]:
import ast

In [ ]:
df = preprocess_data(df)
df.head()


,Sentence_ID,Word,POS,Tag,splited_word
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-GEO, O, O, O, O, O, B-GEO...","['Thousands', 'of', 'demonstrators', 'have', '..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-GPE, O, O, O, O, O, O, O, O, O, O, O, O, O,...","['Iranian', 'officials', 'say', 'they', 'expec..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-TIM, O, O, O, O, O, B-GEO, O, O, O, O...","['Helicopter', 'gunships', 'Saturday', 'pounde..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","['They', 'left', 'after', 'a', 'tense', 'hour-..."
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-GEO, O, O, B-PER, I-PER, O, B-TIM, O, B-GEO...","['U.N.', 'relief', 'coordinator', 'Jan', 'Egel..."


In [ ]:
Tag = df['Tag'].to_list()


In [ ]:
import ast

In [ ]:
flattened_set_train = set(value for sublist in df.Tag.to_list() for value in sublist)

In [ ]:
print(len(flattened_set_train))

17


In [ ]:
target_tag = list(flattened_set_train)
print(target_tag)

['B-EVE', 'B-PER', 'B-GEO', 'O', 'I-ART', 'B-TIM', 'B-GPE', 'B-NAT', 'I-GPE', 'I-NAT', 'I-ORG', 'I-EVE', 'I-PER', 'I-TIM', 'B-ORG', 'I-GEO', 'B-ART']


In [ ]:
num_tags = len(target_tag)
print(num_tags)

17


In [ ]:
tags_map = {tag:i for i,tag in enumerate(flattened_set_train)}
print(tags_map)

{'B-EVE': 0, 'B-PER': 1, 'B-GEO': 2, 'O': 3, 'I-ART': 4, 'B-TIM': 5, 'B-GPE': 6, 'B-NAT': 7, 'I-GPE': 8, 'I-NAT': 9, 'I-ORG': 10, 'I-EVE': 11, 'I-PER': 12, 'I-TIM': 13, 'B-ORG': 14, 'I-GEO': 15, 'B-ART': 16}


In [ ]:
reverse_tag_map = {v:k for k,v in tags_map.items()}

## Encoded tags -- tags with number

In [ ]:
encoded_tags = [[tags_map[w] for w in tag] for tag in Tag]

In [ ]:
df['Word'][0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

In [ ]:
encoded_tags[0]

[0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 12, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0]

## pre processing text

In [ ]:
df['tokenized_word'] = df['Word']

In [ ]:
df.head()

,Sentence_ID,Word,POS,Tag,splited_word,tokenized_word
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-GEO, O, O, O, O, O, B-GEO...","['Thousands', 'of', 'demonstrators', 'have', '...","[Thousands, of, demonstrators, have, marched, ..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-GPE, O, O, O, O, O, O, O, O, O, O, O, O, O,...","['Iranian', 'officials', 'say', 'they', 'expec...","[Iranian, officials, say, they, expect, to, ge..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-TIM, O, O, O, O, O, B-GEO, O, O, O, O...","['Helicopter', 'gunships', 'Saturday', 'pounde...","[Helicopter, gunships, Saturday, pounded, mili..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","['They', 'left', 'after', 'a', 'tense', 'hour-...","[They, left, after, a, tense, hour-long, stand..."
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-GEO, O, O, B-PER, I-PER, O, B-TIM, O, B-GEO...","['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","[U.N., relief, coordinator, Jan, Egeland, said..."


In [ ]:
sentence_list = df['Word'].tolist()

In [ ]:
tokeniser= tf.keras.preprocessing.text.Tokenizer(lower=False,filters='')

tokeniser.fit_on_texts(sentence_list)

In [ ]:
print("Vocab size of Tokeniser ",len(tokeniser.word_index)+1) ## Adding one since 0 is reserved for padding

Vocab size of Tokeniser  35179


In [ ]:
encoded_sentence=tokeniser.texts_to_sequences(sentence_list)
print("original sentence \n",sentence_list[0])
print("encoded sentence \n",encoded_sentence[0])

original sentence 
 ['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
encoded sentence 
 [1114, 4, 1161, 16, 1852, 229, 478, 6, 533, 1, 155, 5, 58, 8, 582, 1, 843, 4, 179, 87, 21, 15, 52, 2]


In [ ]:
max_words = max(len(string.split()) for string in df['splited_word'])
print(max_words)

104


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

max_len = 128

padded_encoded_sentence = pad_sequences(maxlen = max_len,sequences=encoded_sentence,padding = 'post',value=0)
padded_encoded_tags = pad_sequences(maxlen=max_len,sequences=encoded_tags,padding='post',value=tags_map['O'])

print("Shape of Encoded Sentence: ",padded_encoded_sentence.shape)
print("Shape of Encoded Labels: ",padded_encoded_tags.shape)

print("First Encoded Sentence Without Padding:\n",encoded_sentence[0])
print("First Encoded Sentence with padding:\n",padded_encoded_sentence[0])
print("First Sentence Encoded Label without Padding:\n",encoded_tags[0])
print("First Sentence Encoded Label with Padding:\n",padded_encoded_tags[0])


Shape of Encoded Sentence:  (47959, 128)
Shape of Encoded Labels:  (47959, 128)
First Encoded Sentence Without Padding:
 [1114, 4, 1161, 16, 1852, 229, 478, 6, 533, 1, 155, 5, 58, 8, 582, 1, 843, 4, 179, 87, 21, 15, 52, 2]
First Encoded Sentence with padding:
 [1114    4 1161   16 1852  229  478    6  533    1  155    5   58    8
  582    1  843    4  179   87   21   15   52    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
First Sentence Encoded Label without Padding:
 [3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3

In [ ]:
target= [to_categorical(i,num_classes = num_tags) for i in  padded_encoded_tags]
print("Shape of Labels  after converting to Categorical for first sentence: ",target[0].shape)

Shape of Labels  after converting to Categorical for first sentence:  (128, 17)


## Splitting data

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val_test,y_train,y_val_test = train_test_split(padded_encoded_sentence,target,test_size = 0.3,random_state=42)
x_val,x_test,y_val,y_test = train_test_split(x_val_test,y_val_test,test_size=0.2,random_state=42)
print("Input Train Data Shape: ",x_train.shape)
print("Train Labels Length: ",len(y_train))
print("Input Test Data Shape: ",x_test.shape)
print("Test Labels Length: ",len(y_test))

print("Input Validation Data Shape: ",x_val.shape)
print("Validation Labels Length: ",len(y_val))

Input Train Data Shape:  (33571, 128)
Train Labels Length:  33571
Input Test Data Shape:  (2878, 128)
Test Labels Length:  2878
Input Validation Data Shape:  (11510, 128)
Validation Labels Length:  11510


In [ ]:
print("First Sentence in Training Data: ",x_train[0])
print("First sentence Label: ",y_train[0])
print("Shape of First Sentence -Train: ",x_train[0].shape)
print("Shape of First Sentence Label  -Train: ",y_train[0].shape)

First Sentence in Training Data:  [5438   10 2546   48  782  166   26 3902    1   34   10 4674 1270   14
  800  340    2    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
First sentence Label:  [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Shape of First Sentence -Train:  (128,)
Shape of First Sentence Label  -Train:  (128, 17)


In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional,TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=len(tokeniser.word_index)+1,output_dim=32,input_length=max_len))
model.add(Bidirectional(LSTM(units=128,return_sequences=True,recurrent_dropout=0.2)))
model.add(TimeDistributed(Dense(num_tags,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001),metrics=['accuracy'])
model.summary()
checkpointer = ModelCheckpoint(filepath="model.h5", verbose=0, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history=model.fit(x_train,np.array(y_train),batch_size=32,epochs=5,validation_data=(x_val,np.array(y_val)),verbose=1,callbacks=[early_stopping,checkpointer])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 128, 32)           1125728   
                                                                 
 bidirectional_1 (Bidirecti  (None, 128, 256)          164864    
 onal)                                                           
                                                                 
 time_distributed_1 (TimeDi  (None, 128, 17)           4369      
 stributed)                                                      
                                                                 
Total params: 1294961 (4.94 MB)
Trainable params: 1294961 (4.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1050/1050 [==============================] - 771s 729ms/step - loss: 0.1038 - accuracy: 0.9790 - val_loss: 0.0362 - val_accuracy: 0.9904
E

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1050/1050 [==============================] - 758s 722ms/step - loss: 0.0252 - accuracy: 0.9928 - val_loss: 0.0228 - val_accuracy: 0.9934
Epoch 3/5
1050/1050 [==============================] - 754s 718ms/step - loss: 0.0167 - accuracy: 0.9951 - val_loss: 0.0197 - val_accuracy: 0.9942
Epoch 4/5
1050/1050 [==============================] - 727s 693ms/step - loss: 0.0135 - accuracy: 0.9959 - val_loss: 0.0195 - val_accuracy: 0.9943
Epoch 5/5
1050/1050 [==============================] - 746s 710ms/step - loss: 0.0114 - accuracy: 0.9964 - val_loss: 0.0202 - val_accuracy: 0.9943


In [ ]:
preds=model.predict(x_test) ## Predict using model on Test Data

90/90 [==============================] - 8s 81ms/step


In [ ]:
def evaluatePredictions(test_data,preds,actual_preds):
    print("Shape of Test Data Array",test_data.shape)
    y_actual=np.argmax(np.array(actual_preds),axis=2)
    y_pred=np.argmax(preds,axis=2)
    num_test_data=test_data.shape[0]
    print("Number of Test Data Points ",num_test_data)
    data=pd.DataFrame()
    df_list=[]
    for i in range(num_test_data):
        test_str=list(test_data[i])
        df=pd.DataFrame()
        df['test_tokens']=test_str
        df['tokens']=df['test_tokens'].apply(lambda x:tokeniser.index_word[x] if x!=0 else '<PAD>')
        df['actual_target_index']=list(y_actual[i])
        df['pred_target_index']=list(y_pred[i])
        df['actual_target_tag']=df['actual_target_index'].apply(lambda x:reverse_tag_map[x])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tag_map[x])
        df['id']=i+1
        df_list.append(df)
    data=pd.concat(df_list)
    pred_data=data[data['tokens']!='<PAD>']
    accuracy=pred_data[pred_data['actual_target_tag']==pred_data['pred_target_tag']].shape[0]/pred_data.shape[0]


    return pred_data,accuracy

In [ ]:
pred_data,accuracy=evaluatePredictions(x_test,preds,y_test)

Shape of Test Data Array (2878, 128)
Number of Test Data Points  2878


In [ ]:
y_pred=pred_data['pred_target_tag'].tolist()
y_actual=pred_data['actual_target_tag'].tolist()

In [ ]:
pred_data.head(40)

,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
0,4759,From,5,3,B-TIM,O,1
1,521,2004,13,5,I-TIM,B-TIM,1
2,6,to,13,13,I-TIM,I-TIM,1
3,711,2007,13,13,I-TIM,I-TIM,1
4,3,",",3,3,O,O,1
5,1,the,3,3,O,O,1
6,201,economy,3,3,O,O,1
7,2249,grew,3,3,O,O,1
8,67,about,3,3,O,O,1
9,287,10,3,3,O,O,1
